In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import heapq
import geopandas as gpd
import plotly.graph_objects as go
import copy
import geopy
import multiprocessing
import geopy.distance

from joblib import Parallel, delayed
from scipy.optimize import linprog
from collections import Counter
from typing import List, Tuple, Dict, Optional

## Load the data into a dataframe:

In [3]:
roads_df = gpd.read_file('data/ubcv_roads_simple.geojson')

buildings_df = pd.read_csv('data/ubcv_buildings_simple.csv')
otherBuildings_df = pd.read_csv('data/ubcv_poi.csv')
otherBuildings_df["BLDG_CODE"] = otherBuildings_df["PLACENAME"]

df = pd.concat([roads_df, buildings_df], axis=0, ignore_index=True)
buildings_df = buildings_df.drop(columns=["PRIMARY_ADDRESS", "POSTAL_CODE", "CONSTR_STATUS", "BLDG_USAGE"])

otherBuildings_df["NAME"] = otherBuildings_df["PLACENAME"]
otherBuildings_df = otherBuildings_df.drop(columns=["BCN_CLASS", "BCN_LANDUSE", "BCN_CODE", "BCN_NOEMP", "LAST_EDITED_DATE", "LAST_EDITED_USER", "URL", "PHOTOURL", "SERVICE_TYPE", "BL_CLASS", "BL_NAME", "ABBREVIATEDPLACENAME", "PLACENAME2", "CREATED_USER", "LEED", "HTML", "STATUS", "MANAGE", "LICENSE", "HOURS", "PLACENAME", "CREATED_DATE", "CONTACT", "GLOBALID"])

buildings_df = pd.concat([buildings_df, otherBuildings_df], axis=0, ignore_index=True)
buildings_df["IS_BUILDING"] = True

## Helper Functions to Construct the Graph

In [4]:
def calculate_distance(pos1, pos2):
    return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

# Handle edge traces (list of traces or a single trace)
def create_fig(edge_trace=[], node_trace=None, name="", width=1200, height=1200):
    data = []
    if isinstance(edge_trace, list):
        data.extend(edge_trace)
    elif isinstance(edge_trace, (go.Scatter, go.Bar, go.Line)):
        data.append(edge_trace)
    else:
        raise ValueError("edge_trace must be a list of traces or a single valid Plotly trace object.")

    # Handle node trace (single trace)
    if isinstance(node_trace, (go.Scatter, go.Bar, go.Line)): 
        data.append(node_trace)  
    elif node_trace is not None:
        raise ValueError("node_trace must be a single valid Plotly trace object.")

    fig = go.Figure(data=data)

    # Set layout for equal aspect ratio
    fig.update_layout(
        title=name,
        showlegend=False,
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False),
        template="plotly_white",
        width=width,  
        height=height 
    )

    # Make the x and y scales the same to avoid distortion
    fig.update_xaxes(scaleanchor="y", scaleratio=1)
    fig.update_yaxes(scaleanchor="x", scaleratio=1)

    return fig

def create_nodes(node_x, node_y, node_text, mode='markers', color = 'black'):
    if color == 'black':
        color = ['black'] * len(node_text)
        
    node_colors = [
        'red' if color == 'red' else 
        ('black' if text not in buildings_df["BLDG_CODE"].values else 'green')
        for text, color in zip(node_text, color)
    ]
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode=mode,
        marker=dict(
            size=10,
            color= node_colors,
            line=dict(width=2, color=node_colors)
        ),
        text=node_text,
        textposition="top center",
        hoverinfo='text'
    )
    return node_trace

def create_edges(edge_x, edge_y, color='#888', width = 1):
    edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=width, color=color),
    hoverinfo='none',
    mode='lines'
    )
    return edge_trace

## Create Building Graph:

In [5]:
building_G = nx.Graph()
for i in range(len(buildings_df)):
    building_G.add_node(
        buildings_df["BLDG_CODE"][i], 
        pos=(buildings_df["LAT"][i], buildings_df["LONG"][i])
    )

pos = nx.get_node_attributes(building_G, 'pos')
node_x = [x for node, (x, y) in pos.items()]
node_y = [y for node, (x, y) in pos.items()]
node_text = [str(node) for node, (x, y) in pos.items()]

edge_trace = create_edges([], [])    
node_trace = create_nodes(node_x, node_y, node_text)
fig = create_fig(edge_trace, node_trace, name = "Building map, missing roads and edges", width=800, height=1200)

## Create Road Graph:

In [6]:
roads_G = nx.Graph()
listOfCoords = []

# Add nodes with positions
for i in range(len(roads_df)):
    geometry = roads_df["geometry"][i]
    
    if geometry.geom_type == "LineString":
        # Iterate through coordinates in LineString
        for j, (long, lat) in enumerate(geometry.coords):
            if (long, lat) not in listOfCoords:
                if long > -123.2277783865:
                    continue
                listOfCoords.append((long, lat))
                name = roads_df["NAME"][i] if "NAME" in roads_df.columns and pd.notna(roads_df["NAME"][i]) else f"road_{i}"
                node_id = name + f"_point_{j}"  
                roads_G.add_node(
                    node_id,
                    pos=(long, lat)  # Add coordinates as positions
                )
    elif geometry.geom_type == "MultiLineString":
        # Iterate through each LineString in MultiLineString
        for j, line in enumerate(geometry.geoms):  # `geometry.geoms` gives LineString objects
            for k, (long, lat) in enumerate(line.coords):
                if (long, lat) not in listOfCoords:
                    if long > -123.2277783865:
                        continue
                    listOfCoords.append((long, lat))
                    name = roads_df["NAME"][i] if "NAME" in roads_df.columns and pd.notna(roads_df["NAME"][i]) else f"road_{i}"
                    node_id = name + f"_line_{j}_point_{k}"
                    roads_G.add_node(
                        node_id,
                        pos=(long, lat)  # Add coordinates as positions
                    )

pos = nx.get_node_attributes(roads_G, 'pos')

node_x = [x for node, (x, y) in pos.items()]
node_y = [y for node, (x, y) in pos.items()]
node_text = [str(node) for node, (x, y) in pos.items()]

edge_trace = create_edges([], [])
node_trace = create_nodes(node_x, node_y, node_text)
fig = create_fig(edge_trace, node_trace, name = "Road map, missing buildings and edges")

## Merge the Buildings with the Roads Nexus (Preview):

In [7]:
merged_G_noedges = nx.Graph()
for node in roads_G.nodes(data=True):
    merged_G_noedges.add_node(node[0], pos=node[1]['pos'])
for node in building_G.nodes(data=True):
    merged_G_noedges.add_node(node[0], pos=node[1]['pos'])
pos = nx.get_node_attributes(merged_G_noedges, 'pos')
node_x = [x for node, (x, y) in pos.items()]
node_y = [y for node, (x, y) in pos.items()]
node_text = [str(node) for node, (x, y) in pos.items()]
edge_x = []
edge_y = []
edge_trace = create_edges(edge_x, edge_y)
node_trace = create_nodes(node_x, node_y, node_text)
#fig = create_fig(edge_trace, node_trace, name="Combined Building & Road Map, no edges", width=800, height=1200)

## Creating Merged Graph Object (This will likely take a while):

In [8]:
merged_G = nx.Graph()

for node, data in building_G.nodes(data=True):
    merged_G.add_node(node, pos=data['pos'])

for node, data in roads_G.nodes(data=True):
    merged_G.add_node(node, pos=data['pos'])

def calculate_distance(pos1, pos2):
    pos1 = (pos1[1], pos1[0])
    pos2 = (pos2[1], pos2[0])
    return geopy.distance.geodesic(pos1, pos2).meters

buildings = {node for node in merged_G.nodes() if node in buildings_df["BLDG_CODE"].values}
roads = {node for node in merged_G.nodes() if node not in buildings}

NUM_CLOSEST_NODES = 3

def closest_roads_for_building(building):
    building_coord = merged_G.nodes[building]['pos']
    distances = [
        (building, road, calculate_distance(building_coord, merged_G.nodes[road]['pos']))
        for road in roads
    ]
    distances.sort(key=lambda x: x[2])
    return distances[:NUM_CLOSEST_NODES]

building_edges = Parallel(n_jobs=-1)(delayed(closest_roads_for_building)(building) for building in buildings)

for edges in building_edges:
    for building, road, distance in edges:
        merged_G.add_edge(building, road, weight=distance)

def closest_nodes_for_road(road1):
    road1_coord = merged_G.nodes[road1]['pos']
    distances = [
        (road1, node, calculate_distance(road1_coord, merged_G.nodes[node]['pos']))
        for node in merged_G.nodes() if node != road1
    ]
    distances.sort(key=lambda x: x[2])
    return distances[:NUM_CLOSEST_NODES]

road_edges = Parallel(n_jobs=-1)(delayed(closest_nodes_for_road)(road1) for road1 in roads)

for edges in road_edges:
    for road1, node, distance in edges:
        merged_G.add_edge(road1, node, weight=distance)

for road in roads:
    if "line" in road:  # Multiline string
        roadName = road.split("_line_")[0]
        lineNum = int(road.split("_line_")[1].split("_point_")[0])
        pointNum = int(road.split("_point_")[1])
        
        if pointNum > 0:
            prev_point_label = f"{roadName}_line_{lineNum}_point_{pointNum - 1}"
            if prev_point_label in roads:  # Check existence
                weight = calculate_distance(
                    merged_G.nodes[road]['pos'], merged_G.nodes[prev_point_label]['pos']
                )
                merged_G.add_edge(road, prev_point_label, weight=weight)

        next_point_label = f"{roadName}_line_{lineNum}_point_{pointNum + 1}"
        if next_point_label in roads: 
            weight = calculate_distance(
                merged_G.nodes[road]['pos'], merged_G.nodes[next_point_label]['pos']
            )
            merged_G.add_edge(road, next_point_label, weight=weight)
    
    else:  # Single line string
        roadName = road.split("_point_")[0]
        split_road = road.split("_point_")
        if len(split_road) > 1:
            pointNum = int(split_road[1])
        else:
            continue
        
        if pointNum > 0:
            prev_point_label = f"{roadName}_point_{pointNum - 1}"
            if prev_point_label in roads:  # Check existence
                weight = calculate_distance(
                    merged_G.nodes[road]['pos'], merged_G.nodes[prev_point_label]['pos']
                )
                merged_G.add_edge(road, prev_point_label, weight=weight)

        next_point_label = f"{roadName}_point_{pointNum + 1}"
        if next_point_label in roads:
            weight = calculate_distance(
                merged_G.nodes[road]['pos'], merged_G.nodes[next_point_label]['pos']
            )
            merged_G.add_edge(road, next_point_label, weight=weight)

# Creating helper object test_
A = nx.incidence_matrix(merged_G,oriented=True).toarray()
A_shape1 = A.shape[1]
test_A = copy.deepcopy(A)
test_A = np.append(test_A, np.zeros((test_A.shape[0], test_A.shape[1])), axis=1)
for i in range(test_A.shape[0]):
    for j in range(A_shape1):
        if test_A[i][j] == 1: 
            test_A[i][j+A_shape1] = -1
        elif test_A[i][j] == -1:
            test_A[i][j+A_shape1] = 1
        else:
            test_A[i][j+A_shape1] = 0

## Visualize the Merged Graph (Don't run it if you don't want to wait):

In [17]:
combined_pos = nx.get_node_attributes(merged_G, 'pos')

node_x = [x for node, (x, y) in combined_pos.items()]
node_y = [y for node, (x, y) in combined_pos.items()]
node_text = [str(node) for node, (x, y) in combined_pos.items()]

edge_x = []
edge_y = []
for edge in merged_G.edges(data=True):
    x0, y0 = combined_pos[edge[0]]
    x1, y1 = combined_pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

node_trace = create_nodes(node_x, node_y, node_text)
edge_trace = create_edges(edge_x, edge_y)
merged_fig = create_fig(edge_trace, node_trace, name="Merged Graph with Edges", width=800, height=1200)

merged_fig.show()

## Function Implementation of Shortest Path Solved as LP Problem:

In [18]:
nodes = list(merged_G.nodes())
edges = list(merged_G.edges())

# This return a binary decision vector, also reports distance
def shortest_path_LP_basics(building_o, building_d): 
    u_ij=1 
    b = np.zeros(len(nodes))
    b[nodes.index(building_o)] = 1
    b[nodes.index(building_d)] = -1

    c1=list(nx.get_edge_attributes(merged_G, 'weight').values()) 
    c=c1+c1

    result=linprog(c, A_eq=test_A, b_eq=b,bounds=(0, u_ij),method='highs') 
    print("The shortest path has: ", sum(result.x),"edges, with a distance of:", result.fun)
    return result.x

# This return an list of edges, It also reports its foundings
def shortest_path_LP_info(building_o, building_d): 
    u_ij = 1 
    b = np.zeros(len(nodes))
    b[nodes.index(building_o)] = 1
    b[nodes.index(building_d)] = -1

    c1 = list(nx.get_edge_attributes(merged_G, 'weight').values())  
    c = c1 + c1 

    result = linprog(c, A_eq=test_A, b_eq=b, bounds=(0, u_ij), method='highs')
    result_edges = []
    for i in range(len(result.x)):  
            if result.x[i] == 1: 
                edge_index = i % len(edges) 
                result_edges.append(edges[edge_index])
    total_distance = result.fun
    print("The shortest path has:", len(result_edges), "edges, with a distance of:", total_distance)
    return result_edges

# This return an list of edges, without reporting its foundings
def shortest_path_LP(building_o, building_d): 
    u_ij = 1 
    b = np.zeros(len(nodes))
    b[nodes.index(building_o)] = 1
    b[nodes.index(building_d)] = -1

    c1 = list(nx.get_edge_attributes(merged_G, 'weight').values())  
    c = c1 + c1  

    result = linprog(c, A_eq=test_A, b_eq=b, bounds=(0, u_ij), method='highs')
    result_edges = []
    for i in range(len(result.x)):  
            if result.x[i] == 1: 
                edge_index = i % len(edges) 
                result_edges.append(edges[edge_index])
    return result_edges

In [19]:
result1 = shortest_path_LP_basics("CRSW", "LSK")
result2 = shortest_path_LP_info("CRSW", "LSK")
result3 = shortest_path_LP("CRSW", "LSK")
result3

The shortest path has:  43.0 edges, with a distance of: 2594.019284944668
The shortest path has: 43 edges, with a distance of: 2594.019284944668


[('SPRT', 'Gray Ave_point_14'),
 ('MBAH', 'Birney Ave_point_12'),
 ('TENN', 'road_66_point_1'),
 ('TAMK', 'Logan Ln_point_0'),
 ('THR2', 'Thunderbird Blvd_point_12'),
 ('PUMP', 'Education Rd_point_1'),
 ('LSK', 'Sauder Ln_point_4'),
 ('THSP', 'road_216_point_0'),
 ('Neptune Chinese Kitchen', 'Shrum Ln_point_3'),
 ('Hummingbird Child Care', 'Main Mall_point_1'),
 ('Sopron Gate', 'Sopron Ln_point_0'),
 ('Berton Ave_point_8', 'Berton Ave_point_9'),
 ('Berton Ave_point_9', 'Webber Ln_point_13'),
 ('Webber Ln_point_13', 'Webber Ln_point_14'),
 ('Birney Ave_point_12', 'Wesbrook Mall_point_27'),
 ('Main Mall_point_1', 'Main Mall_point_2'),
 ('Education Rd_point_1', 'Education Rd_point_2'),
 ('road_216_point_0', 'road_216_point_1'),
 ('CRSW', 'Gray Ave_point_14'),
 ('SPRT', 'Wesbrook Mall_point_23'),
 ('MBAH', 'Shrum Ln_point_3'),
 ('TENN', 'Logan Ln_point_0'),
 ('TAMK', 'Thunderbird Blvd_point_12'),
 ('THR2', 'Thunderbird Cres_point_5'),
 ('PUMP', 'Sauder Ln_point_3'),
 ('THSP', 'Education Rd

In [20]:
# Separate edges based on the shortest path
search_buildings = ["CRSW", "LSK"]
shortest_path_edges = shortest_path_LP("CRSW", "LSK")

# Separate edge coordinates for edges in the shortest path and the rest
red_edge_x = []
red_edge_y = []
default_edge_x = []
default_edge_y = []

for edge in merged_G.edges():
    x0, y0 = combined_pos[edge[0]]
    x1, y1 = combined_pos[edge[1]]
    if edge in shortest_path_edges or (edge[1], edge[0]) in shortest_path_edges:
        red_edge_x.extend([x0, x1, None])
        red_edge_y.extend([y0, y1, None])
    else:
        default_edge_x.extend([x0, x1, None])
        default_edge_y.extend([y0, y1, None])

# Create edge traces
default_edge_trace = create_edges(default_edge_x, default_edge_y, color='#888')
red_edge_trace = create_edges(red_edge_x, red_edge_y, color='#ff0000', width=5)
                          
# Create a list of colors for each node
node_colors = []
for text in node_text:
    if text in search_buildings:
        node_colors.append('red')  # Set the color to red if the building is in search_buildings
    else:
        node_colors.append('blue')  # Default color is blue

# Create the node trace
node_trace = create_nodes(node_x, node_y, node_text, color=node_colors)

# Create a figure with the updated edge and node traces
fig = create_fig([default_edge_trace, red_edge_trace], node_trace, name="Shortest Path in Red")
fig.show()

## Heuristic Approach for Optimal Meeting Point:

In [21]:
# This one only return all visited building node counted
def heuristic_meeting_point_lp_steps(station_list):
    global merged_G, buildings  
    
    visited_nodes = [] 
    for i in range(len(station_list)):
        for j in range(i + 1, len(station_list)): 
            path_edges = shortest_path_LP(station_list[i], station_list[j])
            if path_edges:
                for edge in path_edges:
                    visited_nodes.extend(edge)  
    node_counts = Counter(visited_nodes)
    return node_counts

# This one return any node as long as it is most visited 
def heuristic_meeting_point_lp(station_list):
    global merged_G 
    visited_nodes = [] 
    for i in range(len(station_list)):
        for j in range(i + 1, len(station_list)):
            path_edges = shortest_path_LP(station_list[i], station_list[j])
            if path_edges:
                for edge in path_edges:
                    visited_nodes.extend(edge)

    node_counts = Counter(visited_nodes)

    heuristic_middle = max(node_counts, key=node_counts.get)

    print("Most visited node (heuristic meeting point):", heuristic_middle)
    return heuristic_middle

# This one only return most visited building node
def heuristic_meeting_point_building_lp(station_list):
    global merged_G, buildings  
    
    visited_nodes = [] 
    for i in range(len(station_list)):
        for j in range(i + 1, len(station_list)): 
            path_edges = shortest_path_LP(station_list[i], station_list[j])
            if path_edges:
                for edge in path_edges:
                    visited_nodes.extend(edge)  
    node_counts = Counter(visited_nodes)

    for node, _ in node_counts.most_common():  
        if node in buildings:  
            print("Most visited building (heuristic meeting point):", node)
            return node

    print("No valid building found. Returning None.")
    return None


In [22]:
source_nodes = ["LSK", "NEST", "UBC Studios", "OSB1", "SAG2", "THUN"]
heuristic_meeting_point_building_lp(source_nodes)

Most visited building (heuristic meeting point): OSB2


'OSB2'

## Functional Implementation of Optimal Common Node

Finds the node in graph G that minimizes the total distance from all source nodes.
    
### Args:
* G (nx.Graph): The input weighted graph.
* source_nodes (List): A list of source nodes.
* weight (str): The edge attribute representing the weight. Default is 'weight'.
        
        
### Returns:
#### Tuple containing:
* optimal_node: The node that minimizes the total distance.
* paths: A dictionary where keys are source nodes and values are the shortest paths to optimal_node.
* distances: A dictionary where keys are source nodes and values are the distances to optimal_node.


In [15]:
def find_optimal_common_node(G, source_nodes, weight="weight"):
    if not source_nodes:
        raise ValueError("The list of source_nodes is empty.")
    
    # Initialize dictionaries to store distances and paths for each source
    all_distances = {}
    all_paths = {}
    
    # Compute shortest paths and distances from each source node
    for source in source_nodes:
        if source not in G:
            raise ValueError(f"Source node {source} is not in the graph.")
        try:
            distances, paths = nx.single_source_dijkstra(G, source, weight=weight)
            all_distances[source] = distances
            all_paths[source] = paths
        except nx.NetworkXNoPath:
            print(f"No paths found from source node {source}.")
            all_distances[source] = {}
            all_paths[source] = {}
    
    # Initialize variables to track the optimal node
    min_total_distance = float('inf')
    optimal_node = None
    
    # Iterate through all nodes to find the one with the smallest total distance
    for node in G.nodes():
        total_distance = 0
        feasible = True
        for source in source_nodes:
            distance = all_distances[source].get(node, None)
            if distance is None:
                # If any source node cannot reach this node, skip it
                feasible = False
                break
            total_distance += distance
        if feasible and total_distance < min_total_distance:
            min_total_distance = total_distance
            optimal_node = node
    
    if optimal_node is None:
        print("No common node found that is reachable from all source nodes.")
        return None, {}, {}
    
    # Retrieve paths and distances for the optimal node
    result_paths = {}
    result_distances = {}
    for source in source_nodes:
        path = nx.shortest_path(G, source, optimal_node, weight=weight)
        distance = all_distances[source][optimal_node]
        result_paths[source] = path
        result_distances[source] = distance
    
    return optimal_node, result_paths, result_distances

In [16]:
source_nodes = ["LSK", "NEST", "MCLD", "UBC Studios", "CRSW", "Gray Ave_point_15", "OSB1", "SAG2"]
optimal_node, paths, distances = find_optimal_common_node(merged_G, source_nodes)
paths

{'LSK': ['LSK',
  'Sauder Ln_point_4',
  'Sauder Ln_point_3',
  'PUMP',
  'Education Rd_point_1',
  'Education Rd_point_2',
  'THSP',
  'road_216_point_0',
  'road_216_point_1',
  'Stores Rd_point_2',
  'road_8_point_3',
  'road_8_point_2',
  'road_8_point_1',
  'road_8_point_0',
  'Hummingbird Child Care',
  'Main Mall_point_1',
  'Main Mall_point_2',
  'Thunderbird Cres_point_11',
  'Sopron Gate',
  'Sopron Ln_point_0',
  'Thunderbird Cres_point_7',
  'Thunderbird Cres_point_5',
  'THR2',
  'Thunderbird Blvd_point_12',
  'TAMK',
  'Logan Ln_point_0'],
 'NEST': ['NEST',
  'road_37_point_3',
  'Body Energy Club ',
  'road_37_point_5',
  'SBME',
  'road_119_point_6',
  'road_119_point_8',
  'WLIB',
  'Hospital Ln_point_3',
  'Patient Park',
  'road_99_point_0',
  'road_135_point_0',
  'Convergent Manufacturing Technologies',
  'Agronomy Rd_point_9',
  'Starbucks TEF3',
  'East Mall_point_18',
  'East Mall_point_15',
  'Thunderbird Blvd_point_13',
  'Logan Ln_point_0'],
 'MCLD': ['MCLD',